#Common tensorflow program structure:

    1. Import and parse the dataset.
    2. Select the type of model.
    3. Train the model.
    4. Evaluate the model's effectiveness.
    5. Use the trained model to make predictions.


#Tensors

A Tensor is a multi-dimensional array. Similar to NumPy `ndarray` objects, `tf.Tensor` objects have a data type and a shape. Additionally, `tf.Tensors` can reside in accelerator memory (like a GPU). TensorFlow offers a rich library of operations (`tf.add, tf.matmul, tf.linalg.inv `etc.) that consume and produce `tf.Tensors`. These operations automatically convert native Python types, for example:

#Layers: common sets of useful operations

Most of the time when writing code for machine learning models you want to operate at a higher level of abstraction than individual operations and manipulation of individual variables.

Many machine learning models are expressible as the composition and stacking of relatively simple layers, and TensorFlow provides both a set of many common layers as a well as easy ways for you to write your own application-specific layers either from scratch or as the composition of existing layers.

TensorFlow includes the full Keras API in the `tf.keras package`, and the Keras layers are very useful when building your own models.



```
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

```



#Model
A model is a relationship between features and the label.
Some simple models can be described with a few lines of algebra, but complex machine learning models have a large number of parameters that are difficult to summarize.

Could you determine the relationship between the four features and the Iris species without using machine learning? That is, could you use traditional programming techniques (for example, a lot of conditional statements) to create a model? Perhaps—if you analyzed the dataset long enough to determine the relationships between petal and sepal measurements to a particular species. And this becomes difficult—maybe impossible—on more complicated datasets. A good machine learning approach determines the model for you. If you feed enough representative examples into the right machine learning model type, the program will figure out the relationships for you.

#Create a model using Keras

The TensorFlow `tf.keras` API is the preferred way to create models and layers. This makes it easy to build models and experiment while Keras handles the complexity of connecting everything together.

The `tf.keras.Sequential` model is a linear stack of layers. Its constructor takes a list of layer instances, and an output layer representing our predictions. The first layer's `input_shape` parameter corresponds to the number of features from the dataset, and is required.

#Steps to build a model "from scratch"

1. Define the model
```
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
    tf.keras.layers.Dense(10, activation=tf.nn.relu),
    tf.keras.layers.Dense(3)
  ])
```
2. Define the loss function

This measures how off a model's predictions are from the desired label, in other words, how bad the model is performing. We want to minimize, or optimize, this value.

```
#Get the loss (predicted vs real) of the current batch
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)
```

3. Define the gradient function


```
#Using gradients (derivatives) get the loss_value 
#(A single value of the 'average error')
#And pass it through the trainable_variables of the model
#Note, trainable_variables is an array of values of each of  your defined layers.

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

```

4. Create an optimizer

An optimizer applies the computed gradients to the model's variables to minimize the loss function. 
```
# To push your variables in the right direction
#Here we use the SGD optimizer, there are plenty to choose from.
#The learning rate is the step size to take for each iteration 'down the hill'
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

```

5. Train the model (Training loop)


```
# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201 #Set number of epochs

for epoch in range(num_epochs): #loop per epoch
  epoch_loss_avg = tf.keras.metrics.Mean() #get the mean of the loss (accumulative)
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy() #define the accuracy

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    loss_value, grads = grad(model, x, y) #get the single loss value and the gradients of each variable
    optimizer.apply_gradients(zip(grads, model.trainable_variables)) #apply the gradients to each variable
    #the optimizer was defined in previous step

    # Track progress
    epoch_loss_avg(loss_value)  # Add current batch loss #you do some function (average) to the loss, it keeps memory of previous values 
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    epoch_accuracy(y, model(x, training=True)) #This also keeps memory of previous values. 

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

```








#Dataset Types:
<ul>
  <li>
    <a>TensorSliceDataset</a> creates a tensor type from numbers

    tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

  </li>
  <li>
    <a>TextLineDatasetV2</a> creates a tensor type from text

    tf.data.TextLineDataset(filename) 
  
  </li>

  <li>
    <a> PrefetchDataset </a> is returned from tf.data.experimental.make_csv_dataset

    train_dataset = tf.data.experimental.make_csv_dataset(train_dataset_fp,
    batch_size, column_names=column_names, label_name=label_name, num_epochs=1)
  </li>
</ul>

#Some useful code snippets/functions:

<ul>
  <li> 
    <a>tf.config.experimental.list_physical_devices("GPU")</a>
    To check if GPU is availiable: 

    print("Is there a GPU available: "),
    print(tf.config.experimental.list_physical_devices("GPU"))
    print("Is the Tensor on GPU #0:  "),
    print(x.device.endswith('GPU:0')) 
  </li>
  <li> 
    <a> a_tensor.shuffle(x).batch(x)</a> Te shufflea y batchea el tensor.

    *Nota,no se que onda con shuffle (https://www.tensorflow.org/api_docs/python/tf/random/shuffle), 
    *batch lo que hace es que te los pone en 'batches' del tamaño indicado
    *Cabe decir que en batch, si no hay información suficiente para que sea un número perfectamente divisible, 
    solo te deja el último batch hasta donde tenga info
  </li>
  <li> 
    <a>layer.variables</a> y <a>layer.trainable_variables </a> 
    inspect all variables (or trainable_variables) in a layer

      the layer is simply a keras layer, example: 
      layer = tf.keras.layers.Dense(10, input_shape=(None, 5))
      # The variables are also accessible through nice accessors
        layer.kernel, layer.bias
      
  Along the same lines: <a> YourBlock.layers </a> returns the layers of the (possibly custom) model

  </li>

  <li> <a> tf.data.experimental.make_csv_dataset </a> When the dataset is a CSV function we use this function into a suitable format. Since this function generates data for training models, the default behavior is to shuffle the data (shuffle=True, shuffle_buffer_size=10000), and repeat the dataset forever (num_epochs=None). We also set the batch_size parameter.
        
    batch_size = 32
    train_dataset = tf.data.experimental.make_csv_dataset(train_dataset_fp,
    batch_size, column_names=column_names, label_name=label_name, num_epochs=1)

  NOTE: this is not enough to start training the data. And usually you have to use a stack function to pass it to <a> MapDataset </a>:

    #Define the stack function 
    def pack_features_vector(features, labels):
      """Pack the features into a single array."""
      features = tf.stack(list(features.values()), axis=1)
      return features, labels 
    pass the function to the dataset
    train_dataset = train_dataset.map(pack_features_vector)

  NOTE #2: for test_datasets you dont have to shuffle it:

    test_dataset = tf.data.experimental.make_csv_dataset(test_fp,batch_size,column_names=column_names,label_name=label_name,num_epochs=1,shuffle=False)

  </li>

  <li><a>tf.convert_to_tensor()</a> This takes a numpy array you give it and makes it a tf.Tensor
      
    predict_dataset = tf.convert_to_tensor([
      [5.1, 3.3, 1.7, 0.5,],
      [5.9, 3.0, 4.2, 1.5,],
      [6.9, 3.1, 5.4, 2.1]
    ])

   </li>
</ul>

#Gradient
https://www.khanacademy.org/math/multivariable-calculus/multivariable-derivatives/gradient-and-directional-derivatives/v/gradient

The gradient is a way of packing togheter all the partial derivative information of a function.

#Notes:

<ul>
  <li>
    For a custom layer or blockModel, after building it:
    
    layer = MyCustom(CustomInputs #In layers its usually output_size,
    in blocks is the input)
    #In this function you get a type <__main__.NameOfCustomClass
  It is important to know that after setting up the function, doing the layer = ....
  you have to activate it?: like so:
  
    _ = layer(tf.zeros([10, 5])) #the thing iside is the input

  </li>
  <li> 
      Gradients: A list of sum(dy/dx) for each x in xs.
      They are an important component of the neural network
  </li>

  <li> just a reminder: softmax returns odds of every output you give it</li>
</ul>